
# Velr Cypher MATCH Cookbook (Movies demo)

This notebook is a **cookbook of MATCH patterns that Velr supports today**, based directly on the
end‑to‑end tests in the `velr-e2e` crate.

The idea is:

- Give you a **copy‑pasteable set of Cypher patterns** to experiment with
- Show what **currently works in Velr** in terms of:
  - Node patterns & labels
  - Relationship patterns (typed, untyped, undirected, multi‑type)
  - Property filters, string predicates, `IN`, `NULL`
  - Cartesian products & multi‑`MATCH`
  - Paths & variable‑length patterns
  - Aggregations, pagination, and functions like `id()`, `type()`, `length()`


In [ ]:
%pip install velr --force-reinstall 
%pip install pandas polars pyarrow --quiet

  Using cached velr-0.1.12-cp313-cp313-macosx_11_0_universal2.whl.metadata (776 bytes)
  Using cached cffi-2.0.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.6 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
Using cached velr-0.1.12-cp313-cp313-macosx_11_0_universal2.whl (1.2 MB)
Using cached cffi-2.0.0-cp313-cp313-macosx_11_0_arm64.whl (181 kB)
Using cached pycparser-2.23-py3-none-any.whl (118 kB)
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.23
    Uninstalling pycparser-2.23:
      Successfully uninstalled pycparser-2.23
  Attempting uninstall: cffi
    Found existing installation: cffi 2.0.0
    Uninstalling cffi-2.0.0:
      Successfully uninstalled cffi-2.0.0
  Attempting uninstall: velr0m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [cffi]
    Found existing installation: velr 0.1.12━━━━━━━━━━━━━━━━━━ 1/3 [cffi]
    Uninstalling velr-0.1.12:0m━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [cffi]
      Successfully uninstalled velr-0.1.12━━━━━━━━━━━━━━

In [2]:
from velr.driver import Velr
import pandas as pd
import polars as pl

db = Velr.open(None)
print("Velr DB opened:", db)


Velr DB opened: <velr.driver.Velr object at 0x1061c1060>


### 0.1 Load Movies CSVs from disk

The Movies demo data is stored as four CSV files in `../data/`:

- `../data/movies_people.csv`
- `../data/movies_movies.csv`
- `../data/movies_directed.csv`
- `../data/movies_acted_in.csv`

We'll load them with **Polars** and bind them into Velr.

In [3]:
# Load Movies demo CSVs from disk.
# Adjust the paths below if your layout differs.
import polars as pl

people_csv    = pl.read_csv("../data/movies_people.csv")
movies_csv    = pl.read_csv("../data/movies_movies.csv")
directed_csv  = pl.read_csv("../data/movies_directed.csv")
acted_in_csv  = pl.read_csv("../data/movies_acted_in.csv")

people_csv.head(), movies_csv.head()


(shape: (5, 7)
 ┌─────────┬────────────────────┬──────┬─────────────────┬──────────┬─────────────┬───────────┐
 │ key     ┆ name               ┆ born ┆ birthplace      ┆ is_actor ┆ is_director ┆ is_writer │
 │ ---     ┆ ---                ┆ ---  ┆ ---             ┆ ---      ┆ ---         ┆ ---       │
 │ str     ┆ str                ┆ i64  ┆ str             ┆ bool     ┆ bool        ┆ bool      │
 ╞═════════╪════════════════════╪══════╪═════════════════╪══════════╪═════════════╪═══════════╡
 │ keanu   ┆ Keanu Reeves       ┆ 1964 ┆ Beirut, Lebanon ┆ true     ┆ false       ┆ false     │
 │ moss    ┆ Carrie-Anne Moss   ┆ 1967 ┆ Burnaby, Canada ┆ true     ┆ false       ┆ false     │
 │ fish    ┆ Laurence Fishburne ┆ 1961 ┆ Augusta, USA    ┆ true     ┆ false       ┆ false     │
 │ weaving ┆ Hugo Weaving       ┆ 1960 ┆ Ibadan, Nigeria ┆ true     ┆ false       ┆ false     │
 │ nolan   ┆ Christopher Nolan  ┆ 1970 ┆ London, UK      ┆ false    ┆ true        ┆ false     │
 └─────────┴─────────────

### 0.2 Bind CSV tables into Velr

Now we bind the Polars DataFrames as in-memory tables so we can `UNWIND BIND(...)` them into nodes and relationships.

In [4]:
# Bind the loaded CSVs into Velr as in-memory tables.
db.bind_polars("_movies_people",   people_csv)
db.bind_polars("_movies_movies",   movies_csv)
db.bind_polars("_movies_directed", directed_csv)
db.bind_polars("_movies_acted_in", acted_in_csv)

print("Bound CSV tables into Velr")


Bound CSV tables into Velr


### 0.3 Create nodes and labels via `UNWIND`

First we create bare `:Person` and `:Movie` nodes, then in a second step
we `SET` extra labels (Actor / Director / Writer, and genre labels).

In [5]:
# Create Person nodes
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (p:Person {
  key:        r.key,
  name:       r.name,
  born:       r.born,
  birthplace: r.birthplace
});
""")

# Add Actor / Director / Writer labels
db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_actor
SET p:Actor;
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_director
SET p:Director;
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_writer
SET p:Writer;
""")

# Create Movie nodes
db.run("""
UNWIND BIND('_movies_movies') AS r
CREATE (m:Movie {
  key:      r.key,
  title:    r.title,
  released: r.released,
  imdb:     r.imdb_id,
  runtime:  r.runtime,
  genres:  [r.genre1, r.genre2]
});
""")

# Add genre labels as in the original script
db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_scifi
SET m:ScienceFiction;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_action
SET m:Action;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_thriller
SET m:Thriller;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_heist
SET m:Heist;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_superhero
SET m:Superhero;
""")

print("Nodes & labels created")


Nodes & labels created


### 0.4 Create DIRECTED and ACTED_IN relationships via `UNWIND`

In [6]:
# DIRECTED
db.run("""
UNWIND BIND('_movies_directed') AS r
MATCH (d:Person {key:r.director_key}), (m:Movie {key:r.movie_key})
CREATE (d)-[:DIRECTED {since:r.since}]->(m);
""")

# ACTED_IN
db.run("""
UNWIND BIND('_movies_acted_in') AS r
MATCH (p:Person {key:r.person_key}), (m:Movie {key:r.movie_key})
CREATE (p)-[:ACTED_IN {
  role:    r.role,
  roles:  [r.role],   // single-element list, like the original seed
  minutes: r.minutes
}]->(m);
""")

print("Relationships created")


Relationships created


## 1. Basic node patterns & labels

In [7]:

# 1.1 All movie titles (single label)
q = """
MATCH (m:Movie)
RETURN m.title AS title
ORDER BY title ASC;
"""
db.to_pandas(q)


,title
0,Inception
1,Memento
2,The Dark Knight Rises
3,The Matrix


In [8]:

# 1.2 Filter on a node property (numeric comparison)
q = """
MATCH (p:Person)
WHERE p.born > 1965
RETURN p.name AS name, p.born AS born
ORDER BY born ASC, name ASC;
"""
db.to_pandas(q)


,name,born
0,Carrie-Anne Moss,1967
1,Lilly Wachowski,1967
2,Christopher Nolan,1970
3,Leonardo DiCaprio,1974
4,Tom Hardy,1977
5,Joseph Gordon-Levitt,1981
6,Elliot Page,1987


In [9]:

# 1.3 Nodes with multiple labels
q = """
MATCH (m:Movie:ScienceFiction)
RETURN m.title AS title, m.released AS released
ORDER BY released ASC, title ASC;
"""
db.to_pandas(q)


,title,released
0,The Matrix,1999
1,Inception,2010


## 2. Relationship patterns (typed, untyped, undirected, multi‑type)

In [10]:

# 2.1 Typed, directed relationship with edge properties in WHERE
q = """
MATCH (a:Actor)-[r:ACTED_IN]->(m:Movie {title:'The Matrix'})
WHERE r.minutes >= 0
RETURN a.name AS actor, r.role AS role, r.minutes AS minutes
ORDER BY minutes DESC, actor ASC;
"""
db.to_pandas(q)


,actor,role,minutes
0,Keanu Reeves,Neo,110
1,Carrie-Anne Moss,Trinity,90
2,Laurence Fishburne,Morpheus,80
3,Hugo Weaving,Agent Smith,75


In [11]:

# 2.2 Typed, directed relationship with edge props in WHERE (another example)
q = """
MATCH (d:Director)-[r:DIRECTED]->(m:Movie)
WHERE r.since >= 2010
RETURN d.name AS director, m.title AS title, r.since AS since
ORDER BY since ASC, title ASC;
"""
db.to_pandas(q)


,director,title,since
0,Christopher Nolan,Inception,2010
1,Christopher Nolan,The Dark Knight Rises,2012


In [12]:

# 2.3 Untyped, directed relationship (any type)
q = """
MATCH (p:Person)-[]->(m:Movie)
RETURN p.name AS person, m.title AS movie
ORDER BY movie ASC, person ASC;
"""
db.to_pandas(q)


,person,movie
0,Christopher Nolan,Inception
1,Elliot Page,Inception
2,Joseph Gordon-Levitt,Inception
3,Leonardo DiCaprio,Inception
4,Tom Hardy,Inception
5,Carrie-Anne Moss,Memento
6,Christopher Nolan,Memento
7,Christopher Nolan,The Dark Knight Rises
8,Tom Hardy,The Dark Knight Rises
9,Carrie-Anne Moss,The Matrix


In [13]:

# 2.4 Untyped, undirected relationship (any type, any direction)
q = """
MATCH (p:Person)--(m:Movie)
RETURN p.name AS person, m.title AS movie
ORDER BY movie ASC, person ASC;
"""
db.to_pandas(q)


,person,movie
0,Christopher Nolan,Inception
1,Elliot Page,Inception
2,Joseph Gordon-Levitt,Inception
3,Leonardo DiCaprio,Inception
4,Tom Hardy,Inception
5,Carrie-Anne Moss,Memento
6,Christopher Nolan,Memento
7,Christopher Nolan,The Dark Knight Rises
8,Tom Hardy,The Dark Knight Rises
9,Carrie-Anne Moss,The Matrix


In [14]:

# 2.5 Multi-type relationship: ACTED_IN or DIRECTED
q = """
MATCH (p:Person)-[r:ACTED_IN|DIRECTED]->(m:Movie)
RETURN p.name AS person, type(r) AS rel_type, m.title AS movie
ORDER BY person ASC, rel_type ASC, movie ASC;
"""
db.to_pandas(q)


,person,rel_type,movie
0,Carrie-Anne Moss,ACTED_IN,Memento
1,Carrie-Anne Moss,ACTED_IN,The Matrix
2,Christopher Nolan,DIRECTED,Inception
3,Christopher Nolan,DIRECTED,Memento
4,Christopher Nolan,DIRECTED,The Dark Knight Rises
5,Elliot Page,ACTED_IN,Inception
6,Hugo Weaving,ACTED_IN,The Matrix
7,Joseph Gordon-Levitt,ACTED_IN,Inception
8,Keanu Reeves,ACTED_IN,The Matrix
9,Lana Wachowski,DIRECTED,The Matrix


In [15]:

# 2.6 Relationship pattern map (edge properties in the pattern)
q = """
MATCH (p:Person)-[:ACTED_IN {role:'Neo'}]->(m:Movie)
RETURN p.name AS name, m.title AS title
ORDER BY title ASC, name ASC;
"""
db.to_pandas(q)


,name,title
0,Keanu Reeves,The Matrix


In [16]:

# 2.7 Node pattern map with multiple fields
q = """
MATCH (m:Movie {title:'The Matrix', released:1999})
RETURN m.title AS title, m.released AS released
ORDER BY title ASC;
"""
db.to_pandas(q)


,title,released
0,The Matrix,1999


## 3. WHERE filters: numeric, boolean logic, ranges

In [17]:

# 3.1 AND + OR on node properties
q = """
MATCH (p:Person)
WHERE p.born > 1965 AND (p.birthplace = 'London, UK' OR p.birthplace = 'Los Angeles, USA')
RETURN p.name AS name, p.born AS born, p.birthplace AS birthplace
ORDER BY birthplace ASC, born ASC, name ASC;
"""
db.to_pandas(q)


,name,born,birthplace
0,Christopher Nolan,1970,"London, UK"
1,Tom Hardy,1977,"London, UK"
2,Leonardo DiCaprio,1974,"Los Angeles, USA"
3,Joseph Gordon-Levitt,1981,"Los Angeles, USA"


In [18]:

# 3.2 Numeric range with >= and <=
q = """
MATCH (m:Movie)
WHERE m.runtime >= 120 AND m.runtime <= 160
RETURN m.title AS title, m.runtime AS runtime
ORDER BY runtime ASC, title ASC;
"""
db.to_pandas(q)


,title,runtime
0,The Matrix,136
1,Inception,148


In [19]:

# 3.3 Cartesian + filter: cross-node comparisons
q = """
MATCH (m1:Movie), (m2:Movie)
WHERE m1.released > m2.released
RETURN m1.title AS newer, m1.released AS newer_year,
       m2.title AS older, m2.released AS older_year
ORDER BY newer_year ASC, newer ASC, older ASC;
"""
db.to_pandas(q)


,newer,newer_year,older,older_year
0,Memento,2000,The Matrix,1999
1,Inception,2010,Memento,2000
2,Inception,2010,The Matrix,1999
3,The Dark Knight Rises,2012,Inception,2010
4,The Dark Knight Rises,2012,Memento,2000
5,The Dark Knight Rises,2012,The Matrix,1999


In [20]:

# 3.4 Cross-alias comparison on properties (Director vs Actor)
q = """
MATCH (d:Director), (a:Actor)
WHERE d.born < a.born
RETURN d.name AS director, d.born AS director_born,
       a.name AS actor, a.born AS actor_born
ORDER BY director_born ASC, director ASC, actor ASC;
"""
db.to_pandas(q)


,director,director_born,actor,actor_born
0,Lana Wachowski,1965,Carrie-Anne Moss,1967
1,Lana Wachowski,1965,Elliot Page,1987
2,Lana Wachowski,1965,Joseph Gordon-Levitt,1981
3,Lana Wachowski,1965,Leonardo DiCaprio,1974
4,Lana Wachowski,1965,Tom Hardy,1977
5,Lilly Wachowski,1967,Elliot Page,1987
6,Lilly Wachowski,1967,Joseph Gordon-Levitt,1981
7,Lilly Wachowski,1967,Leonardo DiCaprio,1974
8,Lilly Wachowski,1967,Tom Hardy,1977
9,Christopher Nolan,1970,Elliot Page,1987



## 4. String predicates, `IN` lists, and `NULL`

These all come from the e2e tests and show what Velr currently supports.


In [21]:

# 4.1 STARTS WITH
q = """
MATCH (m:Movie)
WHERE m.title STARTS WITH 'The'
RETURN m.title AS title
ORDER BY title ASC;
"""
db.to_pandas(q)


,title
0,The Dark Knight Rises
1,The Matrix


In [22]:

# 4.2 CONTAINS on node property
q = """
MATCH (m:Movie)
WHERE m.title CONTAINS 'Knight'
RETURN m.title AS title
ORDER BY title ASC;
"""
db.to_pandas(q)


,title
0,The Dark Knight Rises


In [23]:

# 4.3 ENDS WITH
q = """
MATCH (p:Person)
WHERE p.birthplace ENDS WITH ', UK'
RETURN p.name AS name
ORDER BY name ASC;
"""
db.to_pandas(q)


,name
0,Christopher Nolan
1,Tom Hardy


In [24]:

# 4.4 CONTAINS on birthplace
q = """
MATCH (p:Person)
WHERE p.birthplace CONTAINS 'Los'
RETURN p.name AS name, p.birthplace AS birthplace
ORDER BY name ASC;
"""
db.to_pandas(q)


,name,birthplace
0,Joseph Gordon-Levitt,"Los Angeles, USA"
1,Leonardo DiCaprio,"Los Angeles, USA"


In [25]:

# 4.5 IN list on node property
q = """
MATCH (m:Movie)
WHERE m.released IN [1999, 2010]
RETURN m.title AS title, m.released AS released
ORDER BY released ASC, title ASC;
"""
db.to_pandas(q)


,title,released
0,The Matrix,1999
1,Inception,2010


In [26]:

# 4.6 IN list on relationship property
q = """
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
WHERE a.role IN ['Neo', 'Bane']
RETURN p.name AS name, a.role AS role, m.title AS title
ORDER BY role ASC, name ASC;
"""
db.to_pandas(q)


,name,role,title
0,Tom Hardy,Bane,The Dark Knight Rises
1,Keanu Reeves,Neo,The Matrix


In [27]:

# 4.7 CONTAINS on relationship property
q = """
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
WHERE a.role CONTAINS 'ri'
RETURN p.name AS name, a.role AS role, m.title AS title
ORDER BY role ASC, name ASC;
"""
db.to_pandas(q)


,name,role,title
0,Elliot Page,Ariadne,Inception
1,Carrie-Anne Moss,Trinity,The Matrix


In [28]:

# 4.8 Global OR: predicates over different variables in the same WHERE
q = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.birthplace CONTAINS 'Los' OR m.title STARTS WITH 'The'
RETURN p.name AS name, m.title AS title
ORDER BY title ASC, name ASC;
"""
db.to_pandas(q)


[global_access_expr] ambiguous var 0 in node_ids AND edge_ids; using COALESCE(edge,node)


,name,title
0,Joseph Gordon-Levitt,Inception
1,Leonardo DiCaprio,Inception
2,Tom Hardy,The Dark Knight Rises
3,Carrie-Anne Moss,The Matrix
4,Hugo Weaving,The Matrix
5,Keanu Reeves,The Matrix
6,Laurence Fishburne,The Matrix


In [29]:

# 4.9 IS NULL on a property
q = """
MATCH (p:Person)
WHERE p.birthplace IS NULL
RETURN p.name AS name
ORDER BY name ASC;
"""
db.to_pandas(q)


,name
0,Lana Wachowski
1,Lilly Wachowski


## 5. Paths and variable‑length patterns

In [30]:

# 5.1 Named path + path functions: nodes(), relationships(), length()
q = """
MATCH pth = (p:Person)-[a:ACTED_IN]->(m:Movie)
RETURN
  p.name             AS name,
  m.title            AS title,
  nodes(pth)         AS nodes,
  relationships(pth) AS rels,
  length(pth)        AS hops
ORDER BY title ASC, name ASC;
"""
db.to_pandas(q)


,name,title,nodes,rels,hops
0,Elliot Page,Inception,"[""{\""key\"":\""elliot\"",\""name\"":\""Elliot Page\""...","[""{\""role\"":\""Ariadne\"",\""roles\"":\""[\\\""Ariad...",1
1,Joseph Gordon-Levitt,Inception,"[""{\""key\"":\""jgl\"",\""name\"":\""Joseph Gordon-Le...","[""{\""role\"":\""Arthur\"",\""roles\"":\""[\\\""Arthur...",1
2,Leonardo DiCaprio,Inception,"[""{\""key\"":\""leo\"",\""name\"":\""Leonardo DiCapri...","[""{\""role\"":\""Cobb\"",\""roles\"":\""[\\\""Cobb\\\""...",1
3,Tom Hardy,Inception,"[""{\""key\"":\""hardy\"",\""name\"":\""Tom Hardy\"",\""...","[""{\""role\"":\""Eames\"",\""roles\"":\""[\\\""Eames\\...",1
4,Carrie-Anne Moss,Memento,"[""{\""key\"":\""moss\"",\""name\"":\""Carrie-Anne Mos...","[""{\""role\"":\""Natalie\"",\""roles\"":\""[\\\""Natal...",1
5,Tom Hardy,The Dark Knight Rises,"[""{\""key\"":\""hardy\"",\""name\"":\""Tom Hardy\"",\""...","[""{\""role\"":\""Bane\"",\""roles\"":\""[\\\""Bane\\\""...",1
6,Carrie-Anne Moss,The Matrix,"[""{\""key\"":\""moss\"",\""name\"":\""Carrie-Anne Mos...","[""{\""role\"":\""Trinity\"",\""roles\"":\""[\\\""Trini...",1
7,Hugo Weaving,The Matrix,"[""{\""key\"":\""weaving\"",\""name\"":\""Hugo Weaving...","[""{\""role\"":\""Agent Smith\"",\""roles\"":\""[\\\""A...",1
8,Keanu Reeves,The Matrix,"[""{\""key\"":\""keanu\"",\""name\"":\""Keanu Reeves\""...","[""{\""role\"":\""Neo\"",\""roles\"":\""[\\\""Neo\\\""]\...",1
9,Laurence Fishburne,The Matrix,"[""{\""key\"":\""fish\"",\""name\"":\""Laurence Fishbu...","[""{\""role\"":\""Morpheus\"",\""roles\"":\""[\\\""Morp...",1


In [31]:

# 5.2 Path of length 2 with a named variable
q = """
MATCH pth = (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(p:Person)
RETURN
  m.title     AS title,
  d.name      AS director,
  p.name      AS actor,
  length(pth) AS hops
ORDER BY title ASC, director ASC, actor ASC;
"""
db.to_pandas(q)


,title,director,actor,hops
0,Inception,Christopher Nolan,Elliot Page,2
1,Inception,Christopher Nolan,Joseph Gordon-Levitt,2
2,Inception,Christopher Nolan,Leonardo DiCaprio,2
3,Inception,Christopher Nolan,Tom Hardy,2
4,Memento,Christopher Nolan,Carrie-Anne Moss,2
5,The Dark Knight Rises,Christopher Nolan,Tom Hardy,2
6,The Matrix,Lana Wachowski,Carrie-Anne Moss,2
7,The Matrix,Lana Wachowski,Hugo Weaving,2
8,The Matrix,Lana Wachowski,Keanu Reeves,2
9,The Matrix,Lana Wachowski,Laurence Fishburne,2


In [32]:

# 5.3 WHERE on length(path)
q = """
MATCH pth = (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(p:Person)
WHERE length(pth) = 2
RETURN m.title AS title, d.name AS director, p.name AS actor
ORDER BY title ASC, director ASC, actor ASC;
"""
db.to_pandas(q)


,title,director,actor
0,Inception,Christopher Nolan,Elliot Page
1,Inception,Christopher Nolan,Joseph Gordon-Levitt
2,Inception,Christopher Nolan,Leonardo DiCaprio
3,Inception,Christopher Nolan,Tom Hardy
4,Memento,Christopher Nolan,Carrie-Anne Moss
5,The Dark Knight Rises,Christopher Nolan,Tom Hardy
6,The Matrix,Lana Wachowski,Carrie-Anne Moss
7,The Matrix,Lana Wachowski,Hugo Weaving
8,The Matrix,Lana Wachowski,Keanu Reeves
9,The Matrix,Lana Wachowski,Laurence Fishburne


In [33]:

# 5.4 Variable-length: ACTED_IN with length 1..2 (inclusive)
q = """
MATCH (p:Person)-[:ACTED_IN*1..2]->(m:Movie)
RETURN p.name AS person, m.title AS title
ORDER BY title ASC, person ASC;
"""
db.to_pandas(q)


,person,title
0,Elliot Page,Inception
1,Joseph Gordon-Levitt,Inception
2,Leonardo DiCaprio,Inception
3,Tom Hardy,Inception
4,Carrie-Anne Moss,Memento
5,Tom Hardy,The Dark Knight Rises
6,Carrie-Anne Moss,The Matrix
7,Hugo Weaving,The Matrix
8,Keanu Reeves,The Matrix
9,Laurence Fishburne,The Matrix


In [34]:

# 5.5 Variable-length: exactly 2 hops, any type/direction between Person and Director
q = """
MATCH (p:Person)-[*2]-(d:Director)
RETURN p.name AS person, d.name AS director
ORDER BY director ASC, person ASC;
"""
db.to_pandas(q)


,person,director
0,Carrie-Anne Moss,Christopher Nolan
1,Elliot Page,Christopher Nolan
2,Joseph Gordon-Levitt,Christopher Nolan
3,Leonardo DiCaprio,Christopher Nolan
4,Tom Hardy,Christopher Nolan
5,Tom Hardy,Christopher Nolan
6,Carrie-Anne Moss,Lana Wachowski
7,Hugo Weaving,Lana Wachowski
8,Keanu Reeves,Lana Wachowski
9,Laurence Fishburne,Lana Wachowski


In [35]:

# 5.6 Variable-length: zero-or-more hops, ACTED_IN from Movie to Movie
# In the e2e seed, this collapses to identity pairs because movies have no outgoing ACTED_IN.
q = """
MATCH (m:Movie)-[:ACTED_IN*0..]->(m2:Movie)
RETURN m.title AS left_title, m2.title AS right_title
ORDER BY left_title ASC, right_title ASC;
"""
db.to_pandas(q)


,left_title,right_title
0,Inception,Inception
1,Memento,Memento
2,The Dark Knight Rises,The Dark Knight Rises
3,The Matrix,The Matrix


In [36]:

# 5.7 Variable-length co-actors: exactly 2 ACTED_IN hops between Persons
q = """
MATCH (p:Person)-[:ACTED_IN*2]-(q:Person)
WHERE p.name < q.name
RETURN p.name AS a, q.name AS b
ORDER BY a ASC, b ASC;
"""
db.to_pandas(q)


,a,b
0,Carrie-Anne Moss,Hugo Weaving
1,Carrie-Anne Moss,Keanu Reeves
2,Carrie-Anne Moss,Laurence Fishburne
3,Elliot Page,Joseph Gordon-Levitt
4,Elliot Page,Leonardo DiCaprio
5,Elliot Page,Tom Hardy
6,Hugo Weaving,Keanu Reeves
7,Hugo Weaving,Laurence Fishburne
8,Joseph Gordon-Levitt,Leonardo DiCaprio
9,Joseph Gordon-Levitt,Tom Hardy


In [37]:

# 5.8 Variable-length with path alias and length
q = """
MATCH pt = (p:Person)-[*2]-(d:Director)
RETURN p.name AS person, d.name AS director, length(pt) AS L
ORDER BY director ASC, person ASC, L ASC;
"""
db.to_pandas(q)


,person,director,L
0,Carrie-Anne Moss,Christopher Nolan,1
1,Elliot Page,Christopher Nolan,1
2,Joseph Gordon-Levitt,Christopher Nolan,1
3,Leonardo DiCaprio,Christopher Nolan,1
4,Tom Hardy,Christopher Nolan,1
5,Tom Hardy,Christopher Nolan,1
6,Carrie-Anne Moss,Lana Wachowski,1
7,Hugo Weaving,Lana Wachowski,1
8,Keanu Reeves,Lana Wachowski,1
9,Laurence Fishburne,Lana Wachowski,1


## 6. Aggregates: COUNT, SUM, AVG, MIN, MAX

In [38]:

# 6.1 COUNT(*) over all Movie nodes
q = """
MATCH (m:Movie)
RETURN COUNT(*) AS movies_total;
"""
db.to_pandas(q)


,movies_total
0,4


In [39]:

# 6.2 COUNT(a) per movie (number of ACTED_IN edges)
q = """
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
RETURN m.title AS title, COUNT(a) AS actors
ORDER BY title ASC;
"""
db.to_pandas(q)


,title,actors
0,Inception,4
1,Memento,1
2,The Dark Knight Rises,1
3,The Matrix,4


In [40]:

# 6.3 SUM and AVG over relationship property
q = """
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
RETURN
    m.title AS title,
    SUM(a.minutes) AS minutes_sum,
    AVG(a.minutes) AS minutes_avg
ORDER BY title ASC;
"""
db.to_pandas(q)


,title,minutes_sum,minutes_avg
0,Inception,405,101.25
1,Memento,60,60.00
2,The Dark Knight Rises,75,75.00
3,The Matrix,355,88.75


In [41]:

# 6.4 MIN / MAX over a node property
q = """
MATCH (m:Movie)
RETURN MIN(m.runtime) AS runtime_min, MAX(m.runtime) AS runtime_max;
"""
db.to_pandas(q)


,runtime_min,runtime_max
0,113,164


In [42]:

# 6.5 COUNT over NULLs (property missing -> not counted)
q = """
MATCH (p:Person)
RETURN COUNT(p.nick) AS nick_count;
"""
db.to_pandas(q)


,nick_count
0,0


In [43]:

# 6.6 COUNT over non-NULL runtimes (all movies have runtime in the seed)
q = """
MATCH (m:Movie)
RETURN COUNT(m.runtime) AS runtime_count;
"""
db.to_pandas(q)


,runtime_count
0,4


In [44]:

# 6.7 WHERE before aggregation
q = """
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
WHERE p.born > 1965
RETURN m.title AS title, COUNT(a) AS young_actors
ORDER BY title ASC;
"""
db.to_pandas(q)


,title,young_actors
0,Inception,4
1,Memento,1
2,The Dark Knight Rises,1
3,The Matrix,1


In [45]:

# 6.8 Aggregate by director
q = """
MATCH (d:Director)-[:DIRECTED]->(m:Movie)
RETURN d.name AS director, COUNT(m) AS films
ORDER BY director ASC;
"""
db.to_pandas(q)


,director,films
0,Christopher Nolan,3
1,Lana Wachowski,1
2,Lilly Wachowski,1


## 7. Pagination: `LIMIT` and `SKIP`

In [46]:

# 7.1 LIMIT
q = """
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
RETURN a.name AS actor, m.title AS movie
ORDER BY actor ASC, movie ASC
LIMIT 3;
"""
db.to_pandas(q)


,actor,movie
0,Carrie-Anne Moss,Memento
1,Carrie-Anne Moss,The Matrix
2,Elliot Page,Inception


In [47]:

# 7.2 SKIP + LIMIT
q = """
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
RETURN a.name AS actor, m.title AS movie
ORDER BY actor ASC, movie ASC
SKIP 3 LIMIT 3;
"""
db.to_pandas(q)


,actor,movie
0,Hugo Weaving,The Matrix
1,Joseph Gordon-Levitt,Inception
2,Keanu Reeves,The Matrix


## 8. Multi‑`MATCH` and cartesian products

In [48]:

# 8.1 Two MATCH clauses that form a cartesian product
q = """
MATCH (m1:Movie)
MATCH (m2:Movie)
WHERE m1.released > m2.released
RETURN m1.title AS newer, m2.title AS older
ORDER BY newer ASC, older ASC;
"""
db.to_pandas(q)


,newer,older
0,Inception,Memento
1,Inception,The Matrix
2,Memento,The Matrix
3,The Dark Knight Rises,Inception
4,The Dark Knight Rises,Memento
5,The Dark Knight Rises,The Matrix


In [49]:

# 8.2 Extending a bound variable across MATCH clauses
q = """
MATCH (m:Movie {title:'The Matrix'})
MATCH (p:Person)-[:ACTED_IN]->(m)
RETURN p.name AS actor
ORDER BY actor ASC;
"""
db.to_pandas(q)


,actor
0,Carrie-Anne Moss
1,Hugo Weaving
2,Keanu Reeves
3,Laurence Fishburne


## 9. Functions: `id()`, `type()` and generic patterns

In [50]:

# 9.1 id() and type() on nodes and relationships
q = """
MATCH (p:Person)-[a:ACTED_IN]->(m:Movie)
RETURN
  id(p)   AS pid,
  type(a) AS reltype,
  id(a)   AS aid,
  id(m)   AS mid
ORDER BY mid ASC, pid ASC
LIMIT 10;
"""
db.to_pandas(q)


,pid,reltype,aid,mid
0,1,ACTED_IN,6,12
1,2,ACTED_IN,7,12
2,3,ACTED_IN,8,12
3,4,ACTED_IN,9,12
4,2,ACTED_IN,10,13
5,6,ACTED_IN,11,14
6,7,ACTED_IN,12,14
7,8,ACTED_IN,13,14
8,9,ACTED_IN,14,14
9,9,ACTED_IN,15,15


In [51]:

# 9.2 Match all nodes, return only id(n)
q = """
MATCH (n)
RETURN id(n) AS id
"""
db.to_pandas(q)


,id
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [52]:

# 9.3 Anonymous nodes with a typed relationship, aggregated
q = """
MATCH ()-[a:ACTED_IN]->()
RETURN COUNT(a) AS acted_in_rels;
"""
db.to_pandas(q)


,acted_in_rels
0,10



## 10. Where to go from here

This notebook is **directly grounded in the Velr e2e `MATCH` tests**, so everything here
should reflect what the engine can execute today.

Things you might try next:

- Fork this notebook and **add your own patterns**
- Use it as a **living compatibility matrix** vs. Neo4j-style Cypher
- Swap out the Movies graph for your own domain data and see how the
  patterns behave there

Happy querying! 🧠📈
